# Creating a Normalized Projection for GDP in Africa through 2030

Data:
 - This is a study on using the world development indicators dataset found here: https://www.kaggle.com/worldbank/world-development-indicators. 

Goal:
 - Better understand the types of time series' that represent Africas indicators
 - With our dataset's huge number of indicators (>1000), we want to find k indicators that provide the best representation of the dataset in Africa. By pulling out the k most diverse indicators, we can understand many of the trends that are going on in the Africa, and which indicators have high correlation with each other.
 - We will use the archetype that most closely represents GDP to project a normalized prediction of how GDP will change in the next 10 years.  

Process:
 - Clean and query the data for Africa for indicators where there is 50% or more data density
 - Create a correlation matrix sorted by indicator
 - Use the k-means algorithm to find k cluster centers that best represent the data, minimizing error
 - Match GDP to it's cluster center, and use the center's projection to give a normalized prediction of GDP
 - Display GDP projections in a time series
 
(Conclusion at the end of the study)

In [63]:
#Importing Libraries and World indicator data
import pandas as pd
import numpy as np
import matplotlib as mpl
from sklearn.cluster import KMeans

df = pd.read_csv('WDIData.csv')
print('All World Development Indicator Data:', df.shape)

All World Development Indicator Data: (379368, 66)


In [64]:
#Query only for African nations
africa_codes = ['DZA', 'AGO', 'BEN', 'BWA', 'BFA', 'BDR', 'CMR', 'CPV', 'CAF', 'COM', 'COD', 'DJI', 'EGY', 'GNQ', 'ERI',
                'ETH', 'GAB', 'GMB', 'GHA', 'GIN', 'GNB', 'CIV', 'KEN', 'LSO', 'LBR', 'LBY', 'MDG', 'MWI', 'MLI', 'MRT',
                'MUS', 'MAR', 'MOZ', 'NAM', 'NER', 'NGA', 'COG', 'REU', 'RWA', 'SHN', 'STP', 'SEN', 'SYC', 'SLE', 'SOM',
                'ZAF', 'SSD', 'SDN', 'SWZ', 'TZA', 'TOG', 'TUN', 'UGA', 'ESH', 'ZMB', 'ZWE']

africa_full = df[df['Country Code'].isin(africa_codes)]
print('Shape of Africa:', africa_full.shape)
africa_full.head(3)

Shape of Africa: (73287, 66)


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
70413,Algeria,DZA,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,92.470000,92.830000,93.100000,92.700000,92.620000,NaN,NaN,NaN,NaN,NaN
70414,Algeria,DZA,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,98.764660,99.538582,99.840820,99.931366,99.990051,100.0,100.0,NaN,NaN,NaN
70415,Algeria,DZA,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,97.673732,98.573072,99.479064,99.764565,99.965142,100.0,100.0,NaN,NaN,NaN


In [65]:
#Build some querying equations

#Drop indicators that have data missing based on a given percentage to query only dense data
#Default drops indicators that have 50% or more data missing
def drop_by_density(df, drop_perc=0.5):
    num_years = len(df.columns)-4
    num_countries = len(df['Country Code'].unique())
    na_by_row = df.isna().sum(axis=1)
    indicator_count =  df.groupby('Indicator Name').count().drop(columns = ['Country Name', 'Country Code', 'Indicator Code'])
    keep_these = indicator_count[indicator_count.sum(axis=1) >= (1-drop_perc)*num_years*num_countries].index
    queried = df[df['Indicator Name'].isin(keep_these)]
    return queried 

#Pull out desired indicator by string
def indicator_by_string(df, string_list):
    desired_indicators = df[df['Indicator Name'].str.contains('|'.join(string_list))]
    return desired_indicators

In [66]:
#Choose indicators with desired percentage or more data missing
drop_if_less_than_this_density = .5

africa_fifty = drop_by_density(africa_full, drop_perc=drop_if_less_than_this_density)
print('Number of remaining indicators: ', str(len(africa_fifty['Indicator Name'].unique())))

#Formatting the columns to represent indicators and rows to represent the country and year to prepare for our correlation matrix
africa_fifty = africa_fifty.drop(['Country Code','Indicator Code', 'Unnamed: 65'], axis=1)\
                           .set_index(['Indicator Name','Country Name'])\
                           .stack(level=[0]).unstack(level=[0])
africa_fifty.index.names = ['Country', 'Year']
africa_fifty.head()

Number of remaining indicators:  471


Indicator Name  Adjusted net national income (current US$)  \
Country Year                                                 
Algeria 1970                                  4.047578e+09   
        1971                                  4.323988e+09   
        1972                                  5.673613e+09   
        1973                                  7.087144e+09   
        1974                                  9.631083e+09   

Indicator Name  Adjusted net national income per capita (current US$)  \
Country Year                                                            
Algeria 1970                                           279.819011       
        1971                                           290.742037       
        1972                                           371.164220       
        1973                                           451.128145       
        1974                                           596.387869       

Indicator Name  Adjusted savings: carbon dioxide damage (% of GNI)  \
Country Year                                                         
Algeria 1970                                             0.527300    
        1971                                             0.665356    
        1972                                             0.811346    
        1973                                             0.929302    
        1974                                             0.571352    

Indicator Name  Adjusted savings: carbon dioxide damage (current US$)  \
Country Year                                                            
Algeria 1970                                         2.500437e+07       
        1971                                         3.351074e+07       
        1972                                         5.468055e+07       
        1973                                         8.028553e+07       
        1974                                         7.506401e+07       

Indicator Name  Adjusted savings: consumption of fixed capital (% of GNI)  \
Country Year                                                                
Algeria 1970                                             8.472630           
        1971                                             8.807902           
        1972                                             8.559680           
        1973                                             9.393931           
        1974                                             6.542856           

Indicator Name  Adjusted savings: consumption of fixed capital (current US$)  \
Country Year                                                                   
Algeria 1970                                         4.017686e+08              
        1971                                         4.436109e+08              
        1972                                         5.768782e+08              
        1973                                         8.115729e+08              
        1974                                         8.595976e+08              

Indicator Name  Adjusted savings: education expenditure (% of GNI)  \
Country Year                                                         
Algeria 1970                                             5.230526    
        1971                                             5.230526    
        1972                                             5.230526    
        1973                                             5.230526    
        1974                                             5.230526    

Indicator Name  Adjusted savings: education expenditure (current US$)  \
Country Year                                                            
Algeria 1970                                         2.480294e+08       
        1971                                         2.634360e+08       
        1972                                         3.525104e+08       
        1973                                         4.518825e+08       
        1974        

In [67]:
#Create a correlation matrix for our queried indicators
fifty_corr_matrix = africa_fifty.corr()
fifty_corr_matrix.head(3)

Indicator Name,Adjusted net national income (current US$),Adjusted net national income per capita (current US$),Adjusted savings: carbon dioxide damage (% of GNI),Adjusted savings: carbon dioxide damage (current US$),Adjusted savings: consumption of fixed capital (% of GNI),Adjusted savings: consumption of fixed capital (current US$),Adjusted savings: education expenditure (% of GNI),Adjusted savings: education expenditure (current US$),Adjusted savings: energy depletion (% of GNI),Adjusted savings: energy depletion (current US$),...,"Transport services (% of service imports, BoP)",Travel services (% of commercial service exports),Travel services (% of commercial service imports),"Travel services (% of service exports, BoP)","Travel services (% of service imports, BoP)",Urban population,Urban population (% of total population),Urban population growth (annual %),"Use of IMF credit (DOD, current US$)",Women Business and the Law Index Score (scale 1-100)
Indicator Name,,,,,,,,,,,,,,,,,,,,,
Adjusted net national income (current US$),1.000000,0.255338,0.437392,0.830698,0.009229,0.866471,0.002351,0.831586,0.054487,0.580315,...,-0.060898,-0.038531,0.05048,0.000430,0.057619,0.846201,0.220160,-0.138972,0.626524,0.138760
Adjusted net national income per capita (current US$),0.255338,1.000000,0.347241,0.293129,0.184095,0.262279,0.106690,0.299333,0.103196,0.205078,...,-0.192310,0.105829,0.09343,0.168379,0.110222,0.062797,0.496972,-0.348757,0.216443,0.289509
Adjusted savings: carbon dioxide damage (% of GNI),0.437392,0.347241,1.000000,0.501800,0.239275,0.354481,0.170689,0.462073,0.181097,0.257935,...,-0.232767,0.141500,0.18843,0.198798,0.205577,0.299458,0.428143,-0.235245,0.364233,0.208431


# Clustering Algorithm

For our K-means clustering algorithm, our goal is to create the k cluster centers that each represent an archetype contained in our indicators data set. In order to find the most effective number of cluster centers, k, we will to create an algorithm to measure the overall error of a given number of archetypes, k. We do this by building an objective fuction J, that for each cluster center j, sums the Euclidean distance from each point i in the cluster to its center j, normalized by the cluster standard deviation. 
 $$J=\sum_{j=1}^k\sum_{i=1}^n \frac{\| x_{i}^{(j)}-c_{j}\|}{\sigma_j}$$
 
We want to find the minimum of J, that would show that we've got the best fit for number of cluster centers k.We will run our K means algorithm 100 times for each k, with randomized cluster center location initializations to find the best fit for k. 


Note: We will look for a number of clusters between 4 and 16, so as not to have too many or too few archetypes.

In [ ]:
#Using Gaussian k-means algorithm
for i in range(4,17):
    fifty_corr_matrix